In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

#Dont' forget PNG of datframs
census_data = "../project_one/census_income_data.csv"



In [ ]:
#Obtaining data from csv and api
census_data_income = pd.read_csv(census_data)
census_data_income

# https://enviro.epa.gov/enviro/efservice/SDW_VIOL_ENFORCEMENT/COUNTYSERVED/MCLEAN/STATE/IL/Excel
    
url = "https://enviro.epa.gov/enviro/efservice/SDW_VIOL_ENFORCEMENT/Excel"

response = requests.get(url)
print(response)
